<div style="background-color:#5D73F2; color:#19180F; font-size:40px; font-family:Arial; padding:10px; border: 5px solid #19180F; border-radius:10px"> Emotion Recognition using Deep Emotion </div>


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Importing modules  </div>

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import numpy as np
import cv2
import glob
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Selecting device    </div>

In [ ]:
print(torch.cuda.is_available())
device = torch.device("cuda")

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Inspecting class names    </div>

In [ ]:
dataset_path = "/kaggle/input/fer2013/"
class_names = os.listdir(dataset_path+"/train")
print(class_names)

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Displaying one image for each emotion in train set    </div>

In [ ]:
emotions = []
for file_name in glob.glob(dataset_path+'/train/*/*'):
    emotion = file_name.split('/')[-2]
    if emotion not in emotions:
        img = cv2.imread(file_name)
        plt.imshow(img)
        plt.title("Displaying {} emotion".format(emotion))
        plt.show()
    emotions.append(emotion)

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Visualizing one image for each emotion in val set    </div>

In [ ]:
emotions = []
for file_name in glob.glob(dataset_path+'/test/*/*'):
    emotion = file_name.split('/')[-2]
    if emotion not in emotions:
        img = cv2.imread(file_name)
        plt.imshow(img)
        plt.title("Displaying {} emotion".format(emotion))
        plt.show()
    emotions.append(emotion)

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Defining transforms    </div>

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Resize((48, 48))])


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Creating dataset and dataloader instance    </div>

In [ ]:
train_dataset = ImageFolder(dataset_path+'/train',transform)
train_loader = DataLoader(dataset=train_dataset,batch_size=2048*6)
#creating val data loaders
val_dataset = ImageFolder(dataset_path+'/test',transform)
val_loader = DataLoader(dataset=val_dataset,batch_size=2048)

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Performing sanity check of train loader</div>

In [ ]:
for batch in train_loader:
    print(batch[0].shape,batch[1].shape)
    break


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Ensuring reversed class mappings    </div>

In [ ]:
classes_mappings = train_dataset.class_to_idx
#reversing
reversed_mappings = {v:k for k,v in classes_mappings.items()}
print(reversed_mappings)

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Defining classification base class    </div>

In [ ]:
class FaceEmotionClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        images= images.to(device)
        labels= labels.to(device) 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        images= images.to(device)
        labels= labels.to(device) 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}],  train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
    Defining Deep Emotion architecture based on <a href="https://arxiv.org/abs/1902.01019">paper </a>    </div>

In [ ]:
class Deep_Emotion(FaceEmotionClassificationBase):
    def __init__(self):

        super(Deep_Emotion,self).__init__()
        self.conv1 = nn.Conv2d(3,10,3)
        self.conv2 = nn.Conv2d(10,10,3)
        self.pool2 = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(10,10,3)
        self.conv4 = nn.Conv2d(10,10,3)
        self.pool4 = nn.MaxPool2d(2,2)

        self.norm = nn.BatchNorm2d(10)

        self.fc1 = nn.Linear(810,50)
        self.fc2 = nn.Linear(50,7)

        self.localization = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        self.fc_loc = nn.Sequential(
            nn.Linear(640, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def attention(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 640)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self,input):
        out = self.attention(input)

        out = F.relu(self.conv1(out))
        out = self.conv2(out)
        out = F.relu(self.pool2(out))

        out = F.relu(self.conv3(out))
        out = self.norm(self.conv4(out))
        out = F.relu(self.pool4(out))

        out = F.dropout(out)
        out = out.view(-1, 810)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)

        return out

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Moving tensor to device   </div>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Deep_Emotion().to(device)


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Evaluation and accuracy functions   </div>

In [ ]:
#@torch.no_grad
def evaluate(model,val_loader):
    model.eval()
    outputs =[model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Defining fit function    </div>

In [ ]:
def fit(num_epochs, model, train_loader,val_loader,opt=optimizer):
    history=[]
    for epoch in range(num_epochs):
        model.train()
        train_losses=[]
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
        print("Epoch-{},Loss-{}".format(epoch,loss.item()))

        result = evaluate(model,val_loader)
        result['train_loss'] = sum(train_losses)/len(train_losses)
        model.epoch_end(epoch,result)
        history.append(result)
    
    return history

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Training the model for 50 epochs   </div>

In [ ]:
num_epochs=250
history =fit(num_epochs, model, train_loader,val_loader,opt=optimizer)

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Plotting accuracy and loss curves for val set    </div>

In [ ]:
accuracy = [result['val_acc'] for result in history]
plt.plot(accuracy,'-x')
plt.xlabel('epoch->')
plt.ylabel('accuracy->')
plt.title('Accuracy plot')

In [ ]:
loss = [result['val_loss'] for result in history]
plt.plot(loss,'-x')
plt.xlabel('epoch ->')
plt.ylabel('loss ->')
plt.title('Accuracy plot')

<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Saving the trained model    </div>

In [ ]:
torch.save(model.state_dict(), 'face_emotion_deep_emotion.pth')


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌
Inferencing the trained model   </div>

In [ ]:
test_image = cv2.imread('/kaggle/input/fer2013/test/fear/PrivateTest_25595121.jpg')
plt.imshow(test_image)
plt.show()

In [ ]:
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
test_image = cv2.resize(test_image, (48, 48))
test_tensor = transforms.ToTensor()(test_image)

In [ ]:
mean = [0.485, 0.456, 0.406]  
std = [0.229, 0.224, 0.225] 
test_tensor = transforms.Normalize(mean, std)(test_tensor)


In [ ]:
test_tensor = torch.unsqueeze(test_tensor, 0)


In [ ]:
model.eval()
with torch.no_grad():
    test_tensor = test_tensor.to(device)
    output = model(test_tensor)
    probabilities = F.softmax(output, dim=1)
    predicted_class_index = torch.argmax(probabilities, dim=1).item()
    predicted_class = reversed_mappings[predicted_class_index]
print("Predicted Class is",predicted_class)